# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [92]:
def priest(sex, age, resp_rate, o2_sat, heart_rate, sys_bp, temperature, alertness, inspired_oxygen, performance_status):
    """
    Examples:
    >>> priest('FEMALE', 40, 23, 0.96, 38, 115, 36.3, 'Alert', 'Air', 'Unrestricted normal activity')
    0.09

    >>> priest('Male', 18, 22, 0.95, 105, 115, 36.3, 'Confused or not alert', 'air', 'Unrestricted normal activity')
    0.22

    >>> priest('Unknown', 70, 26, 0.88, 70, 160, 40, 'Awake', 'Non-Invasive', 'Limited self-care') is None
    True
    """
    # Checking for unknown values
    if "Unknown" in (sex, alertness, inspired_oxygen, performance_status):
        return None
    # Age score calculation
    if 16 <= age <= 49:age_score = 0
    elif 50 <= age <= 65:age_score = 2
    elif 66 <= age <= 80:age_score = 3
    else:age_score = 4
    # Respiratory rate score calculation
    if resp_rate < 9:resp_rate_score = 3
    elif 9 <= resp_rate <= 11:resp_rate_score = 1
    elif 12 <= resp_rate <= 20:resp_rate_score = 0
    elif 21 <= resp_rate <= 24:resp_rate_score = 2
    else:resp_rate_score = 3
    # Oxygen saturation score calculation
    o2_sat = o2_sat * 100
    if o2_sat > 95:o2_sat_score = 0
    elif 94 <= o2_sat <= 95:o2_sat_score = 1
    elif 92 <= o2_sat <= 93:o2_sat_score = 2
    else:o2_sat_score = 3
    # Heart rate score calculation
    if heart_rate < 41:heart_rate_score = 3
    elif 41 <= heart_rate <= 50:heart_rate_score = 1
    elif 51 <= heart_rate <= 90:heart_rate_score = 0
    elif 91 <= heart_rate <= 110:heart_rate_score = 1
    elif 111 <= heart_rate <= 130:heart_rate_score = 2
    else:heart_rate_score = 3

    # Systolic blood pressure score calculation
    if sys_bp < 91: sys_bp_score = 3
    elif 91 <= sys_bp <= 100:sys_bp_score = 2
    elif 101 <= sys_bp <= 110:sys_bp_score = 1
    elif 111 <= sys_bp <=219:sys_bp_score = 0
    else:sys_bp_score = 3

    # Temperature score calculation
    if temperature < 35.1:temp_score = 3
    elif 35.1 <= temperature <= 36.0:temp_score = 1
    elif 36.1 <= temperature <= 38.0:temp_score = 0
    elif 38.1 <= temperature <= 39.0:temp_score = 1
    else:temp_score = 2

    # Convert inputs to lowercase for consistent comparison
    alertness = alertness.lower()
    inspired_oxygen = inspired_oxygen.lower()
    performance_status = performance_status.lower()
    sex = sex.lower()
    sex_score = 0 if sex == 'female' else 1
    alertness_score = 0 if alertness == 'alert' else 3
    inspired_oxygen_score = 0 if inspired_oxygen == 'air' else 2

    performance_mapping = {
        'unrestricted normal activity': 0,
        'limited strenuous activity, can do light activity': 1,
        'limited activity, can self-care': 2,
        'limited self-care': 3,
    }

    # Check if performance_status is in the dictionary, and if so, assign the corresponding perf_score
    if performance_status in performance_mapping:
        perf_score = performance_mapping[performance_status]
    else:
        perf_score = 4

    scores = [
        sex_score,
        age_score,
        resp_rate_score,
        o2_sat_score,
        heart_rate_score,
        sys_bp_score,
        temp_score,
        alertness_score,
        inspired_oxygen_score,
        perf_score
    ]

    # Calculate the total PRIEST score
    priest_score = sum(scores)

    score_mapping = [
        (0, 1, 0.01),
        (2, 3, 0.02),
        (4, 4, 0.03),
        (5, 5, 0.09),
        (6, 6, 0.15),
        (7, 7, 0.18),
        (8, 8, 0.22),
        (9, 9, 0.26),
        (10, 10, 0.29),
        (11, 11, 0.34),
        (12, 12, 0.38),
        (13, 13, 0.46),
        (14, 14, 0.47),
        (15, 15, 0.49),
        (16, 16, 0.55),
        (17, 25, 0.59),
        (26, 29, 0.99),
    ]

    for start, end, value in score_mapping:
        if start <= priest_score <= end:
            result = value
            return result



In [93]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('FEMALE', 40, 23, 0.96, 38, 115, 36.3, 'Alert', 'Air', 'Unrestricted normal activity')
Expecting:
    0.09
ok
Trying:
    priest('Male', 18, 22, 0.95, 105, 115, 36.3, 'Confused or not alert', 'air', 'Unrestricted normal activity')
Expecting:
    0.22
ok
Trying:
    priest('Unknown', 70, 26, 0.88, 70, 160, 40, 'Awake', 'Non-Invasive', 'Limited self-care') is None
Expecting:
    True
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [94]:
import requests
def find_hospital(age, gender, risk_percentage):
    """
    Examples:
    >>> find_hospital(19, 'Female', 1.0)
    'Emory Dunwoody Medical Center'
    >>> find_hospital(23, 'female', 0.03)
    'Select Specialty Hospital - Northeast Atlanta'
    >>> find_hospital(56, 'male', 0.5)
    'Emory Dunwoody Medical Center'
    """
    api_base_url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/"
    api_url = f"{api_base_url}?age={age}&sex={gender}&risk_pct={risk_percentage}"

    try:
        response = requests.get(api_url)
        if response.status_code == 200:
            data = response.json()
            return data.get("hospital")
        else:
            print(f"Failed to retrieve data. Status Code: {response.status_code}")
    except Exception as e:
        print(f"An error occurred: {e}")

    return None

In [95]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(19, 'Female', 1.0)
Expecting:
    'Emory Dunwoody Medical Center'
ok
Trying:
    find_hospital(23, 'female', 0.03)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(56, 'male', 0.5)
Expecting:
    'Emory Dunwoody Medical Center'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [96]:
import requests
import json

def get_hospital_address(hospital_name):
    """
    Example:
    >>> get_hospital_address("Emory Dunwoody Medical Center")
    '4500 NORTH SHALLOWFORD ROAD'

    >>> get_hospital_address("Select Specialty Hospital - Northeast Atlanta")
    '1821 CLIFTON ROAD NE'

    >>> get_hospital_address("None") is None
    True
    """
    try:
        # Retrieve the JSON data from the provided URL
        json_base_url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"
        response = requests.get(json_base_url)

        if response.status_code == 200:
            hospital_data = json.loads(response.text)
            hospital_name = hospital_name.upper()

            # Check if the hospital name exists in the JSON data
            if hospital_name in hospital_data:
                hospital_info = hospital_data[hospital_name]
                address = hospital_info.get("ADDRESS")

                formatted_address = f"{address}"
                return formatted_address

        return None

    except Exception as e:
        return 'Error occurred while retrieving hospital data'

import doctest
doctest.run_docstring_examples(get_hospital_address, globals(), verbose=True)


Finding tests in NoName
Trying:
    get_hospital_address("Emory Dunwoody Medical Center")
Expecting:
    '4500 NORTH SHALLOWFORD ROAD'
ok
Trying:
    get_hospital_address("Select Specialty Hospital - Northeast Atlanta")
Expecting:
    '1821 CLIFTON ROAD NE'
ok
Trying:
    get_hospital_address("None") is None
Expecting:
    True
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [97]:
def process_data(file_path):
    try:
        with open(file_path, "r") as file:
            patient_data = file.readlines()
        final_data = {}

        # Extract the headers from the first row
        headers = patient_data[0].strip().split('|')

        for line in patient_data[1:]:
            values = line.strip().split('|')
            patient_number = values[0]
            data = dict(zip(headers, values))

            # Correct capitalization for 'sex' in data
            if 'Sex' in data:
                data['sex'] = data.pop('Sex')

            age = int(data['age'])
            sex = data['sex']

            # Calculate the PRIEST score for the patient
            priest_score = priest(
                sex, age, float(data['breath']), float(data['o2 sat']),
                float(data['heart']), float(data['systolic bp']),
                float(data['temp']), data['alertness'],
                data['inspired'], data['status']
            )

            # Find the hospital based on age, sex, and PRIEST score
            hospital_name = find_hospital(age, sex, priest_score)

            # Get the address of the hospital
            address = get_hospital_address(hospital_name)

            # Create a list with the desired format
            patient_data = [
                sex, age,float(data['breath']),float(data['o2 sat']),float(data['heart']),float(data['systolic bp']),float(data['temp']),data['alertness'],data['inspired'],data['status'],priest_score,hospital_name,address]
            final_data[patient_number] = patient_data
        return final_data
    except Exception as e:
        return None

proceed_patient_data = process_data("people.psv")

if proceed_patient_data:
    with open("proceed_patient_data.json", "w") as file:
        json.dump(proceed_patient_data, file, indent=2)
    print("Results saved to proceed_patient_data.json")
else:
    print("Error occurred while processing the data.")

Results saved to proceed_patient_data.json


## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [98]:
pip install DeepDiff

In [101]:
import requests
import json

# Function to download the  outcomes from the supplied Link to Google Drive
def download_expected_results(gdrive_link, output_file):
    response = requests.get(gdrive_link)
    if response.status_code == 200:
        with open(output_file, "wb") as file:
            file.write(response.content)


expected_results_link = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"
expected_results_file = "/content/people_results.json"


download_expected_results(expected_results_link, expected_results_file)
with open("patient_results.json", "r") as your_results_file:
    your_results = json.load(your_results_file)

with open(expected_results_file, "r") as expected_results_file:
    expected_results = json.load(expected_results_file)

# Make sure your outcomes correspond to what was requried.
if your_results == expected_results:
    print("Your results match the expected results!")
else:
    print("Your results do not match the expected results. Please review your code.")


Your results match the expected results!


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---